In [5]:
import json
from tqdm import tqdm

In [6]:
import requests
key = os.environ['OPENAI_KEY']

def gpt3(prompt):
    headers = {"Authorization": f"Bearer {key}", 'Content-Type': 'application/json',}

    query = {
    'model': 'gpt-3.5-turbo', # Can't use gpt-4 until a payment goes through, I think
        'messages': [
            {
            'role': "user",
            'content': prompt,
            }
        ],
    }

    response = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, json=query)
    return json.loads(response.text)['choices'][0]["message"]["content"]


In [7]:
# Load agg.json
names = {}
with open('../agg.json') as f:
    agg = json.load(f)

    for diagram in tqdm(agg):
        if (diagram['codes'] != None): continue

        repo = diagram['name'].split('_')[0]
        prompt = f'Can you generate a name for this diagram from the {repo} repository? Respond only with the name.'
        prompt += '\n\n'
        prompt += diagram['ascii']

        try:
            name = gpt3(prompt)
        except:
            print(f'Failed to generate name for {diagram["name"]}')
            continue

        names[diagram['name']] = name

 26%|██▌       | 559/2156 [13:06<75:07:33, 169.35s/it]

Failed to generate name for linux_1114


 28%|██▊       | 607/2156 [13:27<06:53,  3.74it/s]    

Failed to generate name for linux_1158


 37%|███▋      | 796/2156 [24:54<66:58:25, 177.28s/it]

...

 70%|██████▉   | 1500/2156 [40:00<04:17,  2.55it/s]    

Failed to generate name for linux_715


 93%|█████████▎| 2008/2156 [53:17<4:40:02, 113.53s/it]

Failed to generate name for llvm_75


100%|██████████| 2156/2156 [54:13<00:00,  1.51s/it]   


In [36]:
# names['chromium_107'] = 'We promise to stop writing C++ like Java'
names['linux_1114'] = 'Padding Chunk Structure'
names['linux_1158'] = 'Audio Communication RPMsg Packet Structure'
names['linux_1328'] = 'Epoll Event Flow Test'
names['linux_590'] = 'IPv4 GRE Tunnel Header Format'
names['linux_715'] = 'ASPEED SoC Multi-function Pin Expression Diagram'
names['llvm_75'] = 'LLVM VOPD Operand Mapping Diagram'

In [37]:
# Write out names.json
# with open('names.json', 'w') as f:
#     json.dump(names, f, indent=2)

In [51]:
import re

def camel_to_snake(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()

def snake_case_to_title_case(s):
    return ' '.join(word.capitalize() for word in s.split('_'))

def any_to_title_case(s):
    return re.sub(' +',  ' ', snake_case_to_title_case(camel_to_snake(s))).title()

In [53]:
names_titles = {}

for [k, v] in names.items():
    names_titles[k] = any_to_title_case(v)

In [55]:
# Write out names.json
with open('names_other.json', 'w') as f:
    json.dump(names_titles, f, indent=2)